# Laplacian Trajectory Editing (LTE)

In this workshop you will be using Learning from Demonstration (LfD) to move a robotic manipulator. LfD is an intuitive way to teach robots new skills, where a demonstrator shows a task which the robot can learn from and generalize to new situations. Today, we will be looking at Laplacian Trajectory Editing, detailed here: https://mediatum.ub.tum.de/doc/1238865/905710.pdf. This method deforms demonstrated trajectories in curvature space such that they maintain their shape. This can be very useful for tasks such as writing, and is applicable to many other tasks. 

The main steps of LTE are as follows:
1. Set up the Laplacian (L) matrix.
2. Use the Laplacian to transform the demonstration into Laplacian space (which represents curvature).
3. Apply constraints and corresponding weights in Laplacian space.
4. Use the inverse transform to get back to the demonstrated coordinate space.

We will go through each of these steps individually.

## Step 1: Set up the Laplacian

The first step is setting up a Laplacian transformation matrix $\boldsymbol{L}$. This matrix, applied to a vector of points, transforms the points into their Laplacian space representation. In the referenced paper, this matrix is defined as

\begin{equation}
\boldsymbol{L} = \frac{1}{2}
\begin{bmatrix}
 2 & -2 &  0 &  0 & \cdots & 0 \\
-1 &  2 & -1 &  0 & \cdots & 0 \\
 0 & -1 &  2 & -1 & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \ddots & \vdots\\
0 & \cdots & 0 & -1 &  2 & -1   \\
0 & \cdots & 0 &  0 & -2 &  2   \\
\end{bmatrix}, 
\end{equation}

where the size of the matrix is determined by the number of points in the given vector. We will fill in the necessary code to create this Laplacian matrix.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

def generate_laplacian(n_pts):
    L = 2.*np.diag(np.ones((n_pts,))) - np.diag(np.ones((n_pts-1,)),1) - np.diag(np.ones((n_pts-1,)),-1)
    L[0,1] = -2.
    L[-1,-2] = -2.
    L = L / 2.
    return L

## Step 2: Transform to Laplacian Space

Next, we will use the generated Laplacian matrix to transform the given demonstration ($\boldsymbol{P}$) to the Laplacian space, where it retains the curvature information. This transformed matrix is denoted as $\boldsymbol{\Delta}$. This transformation is done using the following equation:

\begin{equation}
\boldsymbol{L} \boldsymbol{P} = \boldsymbol{\Delta}
\end{equation}

Please fill out the following function to complete this step:

In [2]:
def laplacian_transform(L, P):
    D = None
    #fill in here
    return D

## Step 3: Append Constraints and Weights

Next we will add constraints and weights in Laplacian space such that when we perform the inverse transform back into the demonstration space, these constraints are met while maintaining the curvature of the demonstration. This allows the LfD representation to perform generalization, an important feature of robot learning. The ability to generalize means that a robot can apply skills to different situations, without having been shown examples in those specific situations. These constraints are applied using concatenation of matrices as

\begin{equation}
\boldsymbol{L}' = 
\begin{bmatrix}
\boldsymbol{L} \\
\boldsymbol{\bar{P}}  \\\end{bmatrix}, 
\end{equation}

and

\begin{equation}
\boldsymbol{\Delta}' = 
\begin{bmatrix}
\boldsymbol{\Delta} \\
\boldsymbol{\bar{C}}  \\\end{bmatrix}, 
\end{equation}

where $\boldsymbol{L}'$ and $\boldsymbol{\Delta}'$ are the concatenated matrices, and $\boldsymbol{\bar{P}}$ are the constrained points and 
$\boldsymbol{\bar{C}}$ are the positions those points are constrained to. Note that these are both multiplied by an arbitrarily high weight $\omega$ such that they are met with more importance. Please fill in the following code for these steps.

In [3]:
fixed_weight = 1e9

def append_constraints(D, C, inds):
    #as an implementation detail, we have C as a list of constraints, not a matrix so we loop over each constraint with a for loop
    D_prime = None
    for const in C:
        pass
        #fill in here
    return D_prime

def append_weights(L, C, inds):
    #here we do the same for loop as above but we care about the indices of the constraints, not the constraints themselves
    L_prime = None
    n_pts = len(L)
    for i in inds:
        pass
        #fill in here
    return L_prime

## Step 4: Append Constraints and Weights

Finally, we will perform the inverse transformation back to the demonstration space. Usually, to perform inverse transformation, the matrix inversion ($\boldsymbol{A}^{-1}$) is used. However, that is only able to be used with square matrices. Since we have concatenated constraints to the $\boldsymbol{L}$ matrix, it is no longer square and we cannot use this method. Therefore, we use the psuedo-inverse (denoted $\boldsymbol{A}^{+}$) instead.

\begin{equation}
\boldsymbol{P}' = \boldsymbol{L}'^+ \boldsymbol{\Delta}'
\end{equation}

Please fill in the following function to complete the pseudo-inverse. Note that the paper recommends solving using a least-squares approach (an optimization which solves this quickly, see `np.linalg.lstsq` for details).

In [4]:
def cartesian_transform(L_prime, D_prime):
    new_traj = None
    #fill in here
    return new_traj

## Putting it all together

Now, we will combine everything together into a single function. Calling this function, which takes a trajectory (a vector of points), as well as constraints (represented by 2 lists: one with points to constrain to, the other with indices for these constrained points), and returns the LTE reproduction.

In [5]:
def LTE(P, C, inds):
    n_pts, n_dims = np.shape(P)
    L = generate_laplacian(n_pts)
    D = laplacian_transform(L, P)
    D_prime = append_constraints(D, C, inds)
    L_prime = append_weights(L, C, inds)
    new_P = cartesian_transform(L_prime, D_prime)
    return new_P

## Testing

We will test LTE with a simulated 2D and 3D trajectory. Try in 2D first and check to make sure it works first. What you should see is the trajectory maintaining the same shape, but deforming in order to meet the given constraints. Once you believe the 2D version is good, try the 3D version and it should work as well! There is no need to modify the code we've written, as it will work in any number of dimensions. Let us know once you are finished.

In [6]:
def main2D():
    # demonstration
    num_points = 50
    t = np.linspace(0, 10, num_points).reshape((num_points, 1))
    x_demo = np.sin(t) + 0.01 * t**2 - 0.05 * (t-5)**2
    y_demo = np.cos(t) - 0.01 * t - 0.03 * t**2
    
    traj = np.hstack((x_demo, y_demo))
    
    new_traj = LTE(traj, [np.array([x_demo[0], y_demo[0]]), np.array([x_demo[-1], y_demo[-1]])], [0, num_points-1])
    
    new_traj2 = LTE(traj, [np.array([x_demo[0]+0.5, y_demo[0]-0.2]), np.array([x_demo[-1], y_demo[-1]])], [0, num_points-1])
    
    new_traj3 = LTE(traj, [np.array([x_demo[0]+0.5, y_demo[0]-0.2]), np.array([x_demo[20]-0.1, y_demo[20]-0.3]), np.array([x_demo[-1], y_demo[-1]+0.2])], [0, 20, num_points-1])
    
    plt.rcParams['figure.figsize'] = (6.5, 6.5)
    fig, axs = plt.subplots(2, 2)
    axs[0][0].set_title('Demonstration')
    axs[1][0].set_title('Same Constraints')
    axs[0][1].set_title('New Initial Point')
    axs[1][1].set_title('New Initial, Final and Viapoint')
    axs[0][0].plot(traj[:, 0], traj[:, 1], 'k', lw=3)
    axs[1][0].plot(traj[:, 0], traj[:, 1], 'k', lw=3)
    axs[0][1].plot(traj[:, 0], traj[:, 1], 'k', lw=3)
    axs[1][1].plot(traj[:, 0], traj[:, 1], 'k', lw=3)
    
    axs[1][0].plot(new_traj[:, 0], new_traj[:, 1], 'm', lw=3)
    axs[1][0].plot(x_demo[0], y_demo[0], 'k.', ms=10)
    axs[1][0].plot(x_demo[-1], y_demo[-1], 'k.', ms=10)
    axs[1][0].plot(x_demo[0], y_demo[0], 'rx', ms=10, mew=2)
    axs[1][0].plot(x_demo[-1], y_demo[-1], 'rx', ms=10, mew=2)
    
    axs[0][1].plot(new_traj2[:, 0], new_traj2[:, 1], 'g', lw=3)
    axs[0][1].plot(x_demo[0], y_demo[0], 'k.', ms=10)
    axs[0][1].plot(x_demo[-1], y_demo[-1], 'k.', ms=10)
    axs[0][1].plot(x_demo[0]+0.5, y_demo[0]-0.2, 'rx', ms=10, mew=2)
    axs[0][1].plot(x_demo[-1], y_demo[-1], 'rx', ms=10, mew=2)
    
    axs[1][1].plot(new_traj3[:, 0], new_traj3[:, 1], 'b', lw=3)
    axs[1][1].plot(x_demo[0], y_demo[0], 'k.', ms=10)
    axs[1][1].plot(x_demo[20], y_demo[20], 'k.', ms=10)
    axs[1][1].plot(x_demo[-1], y_demo[-1], 'k.', ms=10)
    axs[1][1].plot(x_demo[0]+0.5, y_demo[0]-0.2, 'rx', ms=10, mew=2)
    axs[1][1].plot(x_demo[20]-0.1, y_demo[20]-0.3, 'rx', ms=10, mew=2)
    axs[1][1].plot(x_demo[-1], y_demo[-1]+0.2, 'rx', ms=10, mew=2)
    
    plt.show()
    
def main3D():
    # demonstration
    num_points = 50
    t = np.linspace(0, 10, num_points).reshape((num_points, 1))
    x_demo = np.sin(t) + 0.01 * t**2 - 0.05 * (t-5)**2
    y_demo = np.cos(t) - 0.01 * t - 0.03 * t**2
    z_demo = 2 * np.cos(t) * np.sin(t) - 0.01 * t**1.5 + 0.03 * t**2
    
    traj = np.hstack((x_demo, y_demo, z_demo))
    
    new_traj = LTE(traj, [np.array([x_demo[0] + 0.5, y_demo[0] - 0.5, z_demo[0] + 0.5]), np.array([x_demo[-1], y_demo[-1], z_demo[-1]])], [0, num_points-1])
    
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.plot(x_demo.flatten(), y_demo.flatten(), z_demo.flatten(), 'k', lw=3)
    ax.plot(new_traj[:, 0].flatten(), new_traj[:, 1].flatten(), new_traj[:, 2].flatten(), 'm', lw=3)
    ax.plot(x_demo[0], y_demo[0], z_demo[0], 'k.', ms=10)
    ax.plot(x_demo[-1], y_demo[-1], z_demo[-1], 'k.', ms=10)
    ax.plot(new_traj[0, 0], new_traj[0, 1], new_traj[0, 2], 'rx', ms=10, mew=2)
    ax.plot(new_traj[-1, 0], new_traj[-1, 1], new_traj[-1, 2], 'rx', ms=10, mew=2)
    
    plt.show()

In [ ]:
main2D()

In [ ]:
main3D()